In [10]:
import pickle
from pclpy import pcl as pcl2
import pcl
import pcl.pcl_visualization
import time
import numpy as np
import cv2
#import matplotlib
#matplotlib.use('nbagg')
#import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import axes3d, Axes3D #<-- Note the capitalization! 

from open3d import *
import sklearn
from sklearn.cluster import DBSCAN
#import PyQt5
import PyQt4
#from mayavi.mlab import *
from mayavi import mlab

In [6]:
#obtain background frames for background subtraction
filename = 'loco15static.pickle'
file = open(filename, 'rb')
#dataRead = []
#cloud = pcl.PointCloud_PointXYZRGBA()
#visual = pcl.pcl_visualization.CloudViewing()
#time.sleep(2)
#colorFrames = [np.empty((848,480,3)) for i in range(0,6)]
bgFrames = {}
i = 1
while 1:
    try:
        frame = pickle.load(file) #this loads each frame in one at a time, this is where we can process each frame with the transformation matrix
        #cloud = pcl.PointCloud_PointXYZRGBA()
        #cloud2 = pcl.PointCloud_PointXYZRGBA()
        allPoints = np.empty((0,4))
        #for (serial, [poseMat, rmsdValue]) in self.devicesTransformation.items():
        for camera,deviceData in frame.items():
            #depthFrame = deviceData['depth']
            colorFrame = deviceData['infrared']
            if i == 1:
                bgFrames[camera] = colorFrame
            else:
                bgFrames[camera] = cv2.addWeighted(bgFrames[camera],0.5,colorFrame,0.5,0)
            #cameraIntrinsics = deviceData['intrinsics']
            #poseMat = deviceData['poseMat']
            #points = depthFrametoPC(depthFrame, colorFrame, cameraIntrinsics, poseMat)
            #allPoints = np.append(allPoints, points,axis=0)
        #cloud.from_array(allPoints.astype('float32'))

        #visual.ShowColorACloud(cloud)
        print(i)
        i+=1
    except EOFError:
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [62]:
filename = 'loco15walk1.pickle'
file = open(filename, 'rb')
file2 = open('loco15markers.pickle','wb')

#params = cv2.SimpleBlobDetector_Params()
#params.filterByArea = True
#params.maxArea = 50
#params.filterByInertia = False
#params.filterByConvexity = False
#detector = cv2.SimpleBlobDetector_create(params)
#visual = pcl.pcl_visualization.CloudViewing()
#cloud = pcl.PointCloud()
#fig = plt.figure(figsize=(10,12))
#ax = Axes3D(fig)
#fig.show()
#fig.canvas.draw()
#ax.autoscale(False)
#@mlab.animate(delay=100)
#time.sleep(10)
for i in range(1,300):
    frame = pickle.load(file) 
    #j = 0
    #composite1 = np.zeros((480,848))
    #composite2 = np.zeros((480,848))
    markerPoints = np.empty((0,3))
    detectedMarkers = np.empty((0,3))
    for camera, deviceData in frame.items():
        #deviceData = frame['822512060553']
        depthFrame = deviceData['depth']
        infraredFrame = cv2.bitwise_or(deviceData['infrared'],deviceData['infrared'],bgFrames[camera])
        cameraIntrinsics = deviceData['intrinsics']
        poseMat = deviceData['poseMat']
        infraredFrame = cv2.medianBlur(infraredFrame,5)
        infraredFrame[infraredFrame<200]=0

        res,infraredFrameT = cv2.threshold(infraredFrame,200,255,0)
        im2, contours, hierarchy = cv2.findContours(infraredFrameT,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        #blank = np.zeros_like(infraredFrame)
        blank = deviceData['infrared']
        if contours:
            #cv2.drawContours(blank, contours, -1, (255,255,0), 3)
            for cnt in contours:
                c = max(cnt, key=cv2.contourArea)
                ((x, y), radius) = cv2.minEnclosingCircle(c)
                z=depthFrame[int(y),int(x)]/1000
                x1 = (int(x) - cameraIntrinsics['ppx'])/cameraIntrinsics['fx']
                y1 = (int(y) - cameraIntrinsics['ppy'])/cameraIntrinsics['fy'] 
                depth = np.array([z*x1,z*y1,z,1])
                depth = np.array(np.matmul(poseMat, depth)[0:3])
                markerPoints=np.vstack((markerPoints,depth))
                #M = cv2.moments(c)
                #try:
                #    cx = int(M['m10']/M['m00'])
                #    cy = int(M['m01']/M['m00'])
                #except:
                #    pass
                #center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                #if y < 400:
                #code to show each cameras image in new window
                cv2.circle(blank, (int(x),int(y)), 2, (255,255,0), 3)
        if camera == '822512060553':
            cv2.imshow('822512060553',blank)
            cv2.waitKey(1)
        elif camera == '823112060874':
            cv2.imshow('823112060874',blank)
            cv2.waitKey(1)
        elif camera == '822512060853':
            cv2.imshow('822512060853',blank)
            cv2.waitKey(1)
        elif camera == '823112060112':
            cv2.imshow('823112060112',blank)
            cv2.waitKey(1)
        elif camera == '822512061105':
            cv2.imshow('822512061105',blank)
            cv2.waitKey(1)
        elif camera == '822512060522':
            cv2.imshow('822512060522',blank)
            cv2.waitKey(1)
            #print(x,y,radius)
                
    db = DBSCAN(eps=0.0300, min_samples=2).fit(markerPoints)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    for label in set(labels):
        if label != -1:
            class_member_mask = (labels == label)
            marker = markerPoints[class_member_mask & core_samples_mask]
            detectedMarkers = np.vstack((detectedMarkers, np.mean(marker,axis=0)))
    pickle.dump(detectedMarkers, file2)
    print(i,detectedMarkers)
    j+=1
    #plt.mlab_source.reset(x=detectedMarkers[:,0],y=detectedMarkers[:,1],z=detectedMarkers[:,2])
    #mlab.show()
    #yield
    #plt.mlab.points3d(detectedMarkers[:,0],detectedMarkers[:,1],detectedMarkers[:,2])
    #mlab.show()
    #print(detectedMarkers)
    #ax.clear()
    #ax.scatter(detectedMarkers[:,0],detectedMarkers[:,1],detectedMarkers[:,2])
    #fig.canvas.draw()
    #cloud.from_array(detectedMarkers.astype('float32'))
    #visual.ShowMonochromeCloud(cloud)
    #visual.addSphere()
    #keypoints = detector.detect(infraredFrame)
    #im2, contours, hierarchy = cv2.findContours(infraredFrame,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    #print(keypoints)
    #inff = cv2.cvtColor(np.asanyarray(infraredFrame), cv2.COLOR_GRAY2RGB)

    #infraredKeypoints = cv2.drawKeypoints(np.zeros_like(inff), keypoints, np.array([]), (0,255,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    #finalComposite = np.vstack((composite1,composite2))
    #cv2.imshow('im',finalComposite)
    #cv2.waitKey(1)
    #plt.figure(figsize=(12,15))
    #plt.imshow(np.vstack((blank,infraredFrame)))
    #plt.show()
'''
822512060553
823112060874
822512060853
823112060112
822512061105
822512060522
'''

cv2.destroyAllWindows()
file.close()


1 [[-0.33520903 -0.15118574  0.01349275]
 [ 0.04138559  0.11040356 -0.05498039]
 [ 0.1166837   0.01360166 -0.02394088]
 [ 0.0436741   0.11036534 -0.11066618]
 [ 0.55400737  0.30721933  0.0044677 ]
 [ 0.12235116  0.13487486 -0.03068537]
 [ 0.1675471   0.07674956 -0.01711563]
 [ 0.09011401  0.17368957 -0.10061101]
 [ 0.10989773  0.07031136 -0.06497475]]
2 [[ 0.55972693  0.30330029  0.00414545]
 [ 0.16079848  0.08896559 -0.01666143]
 [ 0.12685447  0.02599994 -0.03167004]
 [ 0.11891831  0.1492419  -0.03226092]
 [ 0.10449296  0.08205917 -0.06775547]
 [-0.33634714 -0.15137003  0.01380075]
 [ 0.09261871  0.02798366 -0.0165431 ]
 [ 0.03636243  0.12310709 -0.05764177]
 [ 0.03977449  0.12300033 -0.11316635]
 [ 0.08583627  0.18515918 -0.10510183]]
3 [[-0.3351322  -0.15130565  0.01363267]
 [ 0.10256582  0.04201437 -0.02584373]
 [ 0.03220584  0.13311692 -0.06340255]
 [ 0.03194015  0.13787507 -0.11729609]
 [ 0.52972831  0.31965716  0.00417037]
 [ 0.58423077  0.29251387  0.00150494]
 [ 0.11281876  0.

25 [[ 5.39527340e-01  3.23140328e-01  2.81354494e-03]
 [ 2.02602693e-01  6.58600957e-02 -1.10035818e-01]
 [-3.33585483e-01 -1.51688072e-01  1.41793805e-02]
 [-3.56576941e-01 -6.55205884e-02  1.40174151e-02]
 [ 1.41167040e-01  2.11452154e-03 -1.86379451e-01]
 [ 1.79392664e-01  6.94277518e-02 -1.92105636e-01]
 [ 2.36331325e-01 -3.16270834e-04 -6.05086241e-02]]
26 [[-0.33438146 -0.15292657  0.01415188]
 [-0.34146108 -0.0810927   0.01466333]
 [ 0.54118492  0.32088313  0.00458438]
 [ 0.21651224  0.04047419 -0.10214119]
 [ 0.19684507 -0.0135269  -0.12109331]
 [ 0.15308986 -0.02080442 -0.17658872]
 [ 0.25374303 -0.03439576 -0.0525815 ]
 [ 0.19900518  0.04229575 -0.18130715]]
27 [[-0.33450962 -0.15282626  0.01375204]
 [-0.34554826 -0.08101141  0.01391108]
 [ 0.54118492  0.32088313  0.00458438]
 [ 0.24251542 -0.0062244  -0.0746621 ]
 [ 0.20630021  0.01864396 -0.16619873]]
28 [[ 0.26529026 -0.04286485 -0.05157601]
 [ 0.24167204 -0.00597795 -0.08714027]
 [ 0.21372098  0.01588655 -0.16574916]
 [-0

67 [[-0.33880602 -0.15079782  0.01381412]
 [ 0.25505954 -0.18977953 -0.0711615 ]
 [ 0.39999656 -0.30577466 -0.01405728]
 [ 0.56352166  0.30594998  0.00327147]
 [ 0.35366713 -0.24351313 -0.01722202]
 [ 0.31999623 -0.20276763 -0.06921493]
 [ 0.26639644 -0.25996833 -0.0834254 ]
 [ 0.35323355 -0.36644691 -0.02617916]
 [ 0.27097961 -0.27866351 -0.02992617]
 [-0.3477044  -0.09759996  0.01155308]]
68 [[ 0.38982898 -0.29696572 -0.01271595]
 [ 0.3487085  -0.23209102 -0.01522078]
 [ 0.31464269 -0.19108853 -0.07056511]
 [ 0.35219835 -0.35702707 -0.02865729]
 [ 0.26092633 -0.24651259 -0.08034498]
 [-0.33754191 -0.15025228  0.01319792]
 [-0.34332209 -0.0965306   0.01220756]
 [ 0.56278652  0.31007523  0.00410678]]
69 [[-0.34008125 -0.13069655  0.01315214]
 [ 0.25599618 -0.24909724 -0.03194087]
 [ 0.56117282  0.30746787  0.00312071]
 [ 0.79924161 -0.1069455  -0.47471498]
 [ 0.38247028 -0.28233234 -0.01080506]
 [ 0.30543017 -0.17486207 -0.06962103]
 [ 0.3392917  -0.21835642 -0.0186918 ]
 [ 0.32495428 

92 [[-0.33304263 -0.15219682  0.01401745]
 [ 0.17324673 -0.04721632 -0.01913581]
 [ 0.16799652  0.00568357 -0.06308334]
 [ 0.10123666  0.03773449 -0.04426671]
 [ 0.10139163  0.04538288 -0.10054306]
 [ 0.58046392  0.28701089  0.00228989]
 [ 0.53022737  0.31908954  0.00384345]
 [ 0.22645419  0.01889091 -0.01889768]
 [ 0.18200624  0.07727267 -0.02512669]
 [ 0.13924036  0.11965651 -0.08786484]]
93 [[-0.33544009 -0.15170818  0.01397165]
 [ 0.1744559  -0.03337243 -0.02435263]
 [ 0.15851919  0.0177356  -0.06458475]
 [ 0.09404278  0.05973837 -0.10240755]
 [ 0.09578361  0.0540582  -0.04617689]
 [ 0.52968301  0.3195677   0.00387618]
 [ 0.58230921  0.28802406  0.00160646]
 [ 0.1743733   0.08954778 -0.02554196]
 [ 0.22089445  0.03267306 -0.01774324]
 [ 0.13391803  0.13035295 -0.09106538]]
94 [[-0.33660107 -0.15126029  0.01391189]
 [ 0.16219753 -0.02395459 -0.0225064 ]
 [ 0.14992664  0.03838125 -0.06606623]
 [ 0.08641156  0.0713501  -0.0504371 ]
 [ 0.09017108  0.06869044 -0.10462241]
 [ 0.21151449 

115 [[-0.35618481 -0.06510458  0.01373824]
 [-0.33758974 -0.15148017  0.0139218 ]
 [ 0.14037196  0.06942178 -0.06547007]
 [ 0.11204285  0.10939772 -0.16428918]
 [ 0.12567798  0.08755927 -0.21385289]
 [ 0.19786134  0.12025945 -0.07833466]
 [ 0.17950693  0.15752247 -0.1347746 ]
 [ 0.16952762  0.15348855 -0.22733451]
 [ 0.54596932  0.3226223   0.00475155]]
116 [[-0.35233607 -0.0779307   0.01490028]
 [-0.33471179 -0.15180665  0.01396538]
 [ 0.15390754  0.04841509 -0.06348158]
 [ 0.1306008   0.07758642 -0.21494156]
 [ 0.20651755  0.11084047 -0.08286268]
 [ 0.19127079  0.14049871 -0.13472794]
 [ 0.17781434  0.13773605 -0.22571883]
 [ 0.5503886   0.31962362  0.00614939]
 [ 0.12412336  0.08506225 -0.16129695]
 [ 0.1287055   0.11301286 -0.25042473]]
117 [[ 0.21879753  0.09305537 -0.08097188]
 [ 0.20081605  0.12497676 -0.13346241]
 [ 0.18361176  0.12379323 -0.2224122 ]
 [-0.33461552 -0.15063637  0.01380874]
 [ 0.18008325  0.02285751 -0.06189691]
 [ 0.18448504  0.05040361 -0.13243966]
 [-0.356752

...

179 [[-0.33433062 -0.15165767  0.014786  ]
 [-0.34777109 -0.08868053  0.01442237]
 [ 0.17248727  0.0041858  -0.0824728 ]
 [ 0.18362643 -0.06595095 -0.08831116]
 [ 0.2548066  -0.16591181 -0.02231699]
 [ 0.24706968 -0.11298522 -0.05617008]
 [ 0.52095529  0.32557485  0.00271284]
 [ 0.5824168   0.28739631  0.00166292]
 [ 0.26204276 -0.03812784 -0.01920306]
 [ 0.30583696 -0.09813054 -0.01576676]
 [ 0.22156524  0.00603585 -0.07642851]]
180 [[-0.33899785 -0.08122826  0.01491809]
 [ 0.24064707 -0.0966114  -0.05863499]
 [ 0.17859204 -0.06714868 -0.03503978]
 [-0.3335691  -0.15220703  0.0143278 ]
 [ 0.16371561  0.01528923 -0.08384678]
 [ 0.56445845  0.30165192  0.00156224]
 [ 0.25553651 -0.02711301 -0.01992913]
 [ 0.29894153 -0.08643642 -0.01605687]
 [ 0.21423297  0.01695974 -0.07841162]
 [ 0.26045541 -0.15228921 -0.03014863]]
181 [[ 0.24384845 -0.1462016  -0.02436   ]
 [ 0.23412489 -0.08827016 -0.05782894]
 [ 0.17282595 -0.05464947 -0.03440537]
 [ 0.16821476 -0.042021   -0.09191587]
 [-0.334772

200 [[-0.3310489  -0.15304536  0.01429573]
 [-0.33931615 -0.08137025  0.01275684]
 [ 0.1140727   0.09863723 -0.02422709]
 [ 0.13043392  0.13517458 -0.08537456]
 [ 0.07030613  0.17417354 -0.10147409]
 [ 0.08136343  0.16433083 -0.15564115]
 [ 0.15970366  0.17194516 -0.02503608]
 [ 0.13950403  0.21894152 -0.0705625 ]
 [ 0.13051326  0.23013018 -0.1642871 ]
 [ 0.54026106  0.32240333  0.00314025]
 [ 0.59416626  0.28337987 -0.00172127]]
201 [[ 0.56127851  0.31188773  0.01012202]
 [ 0.15589219  0.1821514  -0.03234179]
 [ 0.12536516  0.14622617 -0.09487623]
 [-0.34573402 -0.0786906   0.01190113]
 [-0.33336423 -0.154377    0.01419319]
 [ 0.08111351  0.13184493 -0.02032047]
 [ 0.07302889  0.18562364 -0.12605376]
 [ 0.0887016   0.1630031  -0.17450182]
 [ 0.13844694  0.22652784 -0.09089386]
 [ 0.1367536   0.22810306 -0.1808107 ]
 [ 0.11369888  0.12826053 -0.03240022]
 [ 0.52338615  0.31979377  0.00180036]]
202 [[ 0.1511371   0.18630889 -0.03912377]
 [ 0.14087508  0.22466972 -0.10025756]
 [ 0.144388

222 [[ 0.37065194 -0.15157359 -0.04763965]
 [ 0.34375194 -0.19454908 -0.08697969]
 [-0.35635222 -0.064657    0.01414857]
 [-0.33581835 -0.15461879  0.01412196]
 [ 0.26213715 -0.08016729 -0.14728228]
 [ 0.32035934 -0.09394582 -0.12716829]
 [ 0.33042522  0.69636328 -0.43442986]
 [ 0.52357327  0.31943882  0.00221515]
 [ 0.59374093  0.28436309 -0.0021502 ]]
223 [[-0.35625193 -0.0653672   0.0145141 ]
 [-0.33344265 -0.15203772  0.01433336]
 [ 0.27698647 -0.10072978 -0.13450528]
 [ 0.54204952  0.32005594  0.00437127]
 [ 0.40667083 -0.21264012 -0.03150444]
 [ 0.3750762  -0.15832192 -0.04926845]
 [ 0.33526059 -0.11394774 -0.12045102]
 [ 0.59459128  0.2833079  -0.00174013]]
224 [[-0.3332442  -0.15373876  0.01465601]
 [ 0.35604632 -0.22664834 -0.08043566]
 [ 0.42103893 -0.24494158 -0.02734852]
 [ 0.53947013  0.32296072  0.00281477]
 [ 0.39165187 -0.18321131 -0.04568001]
 [-0.35625193 -0.0653672   0.0145141 ]]
225 [[-0.35613137 -0.06496659  0.01455526]
 [-0.33352872 -0.15208694  0.01366918]
 [ 0.4

259 [[ 0.39192521 -0.25321907 -0.0163358 ]
 [ 0.35147926 -0.19025618 -0.01314236]
 [ 0.31505326 -0.147288   -0.06937846]
 [ 0.32627013 -0.31079114 -0.01646798]
 [ 0.32565822 -0.24328735 -0.05917041]
 [ 0.2622197  -0.20968634 -0.03430051]
 [-0.33392416 -0.15164029  0.01387315]
 [ 0.2601736  -0.1999514  -0.08678765]
 [-0.35065739 -0.08986011  0.01297473]
 [ 0.5485663   0.32065017  0.00558797]]
260 [[ 0.38205729 -0.24150329 -0.01084974]
 [ 0.34209402 -0.17202495 -0.01509666]
 [ 0.30712624 -0.13003748 -0.06942505]
 [ 0.3233897  -0.29942433 -0.01934717]
 [-0.33518448 -0.15178215  0.01398639]
 [-0.34351806 -0.08759885  0.0132694 ]
 [ 0.25048628 -0.18246051 -0.08814496]
 [ 0.54680882  0.32289262  0.00433412]]
261 [[ 0.54787542  0.3223499   0.00439523]
 [ 0.33604607 -0.16227631 -0.01526235]
 [ 0.37776702 -0.22690789 -0.01485546]
 [ 0.29925349 -0.11431372 -0.07079277]
 [ 0.3154618  -0.2847866  -0.02006218]
 [-0.33528849 -0.15201059  0.01420731]
 [ 0.31247668 -0.22078626 -0.06137396]
 [-0.350420

283 [[ 0.21549719  0.07220544 -0.0174298 ]
 [ 0.17713855  0.12547855 -0.03139855]
 [ 0.15636647  0.01761357 -0.02210634]
 [ 0.14177118  0.16661378 -0.10333331]
 [ 0.15517741  0.07102533 -0.06858215]
 [-0.35623323 -0.06423751  0.01420042]
 [-0.33146996 -0.15119276  0.01412143]
 [ 0.09017598  0.11074905 -0.05840503]
 [ 0.08529377  0.17415395 -0.11758135]
 [ 0.09368722  0.11204385 -0.11545684]
 [ 0.52965033  0.319467    0.00406181]
 [ 0.58407129  0.27818264  0.00061106]]
284 [[ 0.21223381  0.08241728 -0.01747284]
 [ 0.16613821  0.14235389 -0.03414861]
 [ 0.14934222  0.03051711 -0.02210555]
 [ 0.13871561  0.18173503 -0.10988052]
 [ 0.14940209  0.08249589 -0.0711763 ]
 [-0.3362291  -0.15338508  0.0140092 ]
 [-0.35662369 -0.06464352  0.01448225]
 [ 0.08571764  0.122845   -0.06164263]
 [ 0.08973322  0.12227335 -0.11974141]
 [ 0.07943595  0.18803359 -0.12135551]
 [ 0.5827096   0.28645923  0.00175732]
 [ 0.52774399  0.31015903  0.00495238]]
285 [[-0.33461075 -0.15371094  0.01425974]
 [ 0.144300

In [61]:
DBSCAN(eps=0.0300, min_samples=2).fit(markerPoints).labels_

array([ 0,  1,  2,  3,  4,  4,  4,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  5,  6,  7,  8,  8,  8,  0,  4,  3,  5,  5, -1,
        6,  7,  7,  8,  8,  8,  8,  1,  1,  2, -1], dtype=int64)

In [41]:
z=depthFrame[int(y),int(x)]/1000
x1 = (int(x) - cameraIntrinsics['ppx'])/cameraIntrinsics['fx']
y1 = (int(y) - cameraIntrinsics['ppy'])/cameraIntrinsics['fy']
print(z,x1*z,y1*z)

0.552 0.31276389704622237 0.1702270304498491


In [17]:
seedPoint = movingMarkers[0]
plt = mlab.points3d(seedPoint[:,0],seedPoint[:,1],seedPoint[:,2])
i = 0
@mlab.animate(delay=100)
def anim():
    for i in range(1,299):
        point = movingMarkers[i]
        plt.mlab_source.reset(x=point[:,0],y=point[:,1],z=point[:,2])
        yield
anim()
mlab.show()

IndexError: list index out of range

In [36]:
depthFrame.flatten()[int(x)*int(y)]

573

In [182]:
allPoints = getDepthandMarkerPoints(infraredFrame,depthFrame,deviceData['intrinsics'],deviceData['poseMat'])
markerPoints = allPoints[allPoints[:,3]>0]
mpcd = PointCloud()
mpcd.points=Vector3dVector(markerPoints[:,0:3])
mpcd.colors=Vector3dVector(markerPoints[:,3:6])
draw_geometries([mpcd])

In [7]:
def getDepthandMarkerPoints(infraredFrame,depthFrame,cameraIntrinsics,poseMat):
    infraredFrame = cv2.medianBlur(infraredFrame,5)
    infraredFrame[infraredFrame<215]=0

    res,infraredFrameT = cv2.threshold(infraredFrame,200,255,0)
    im2, contours, hierarchy = cv2.findContours(infraredFrameT,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    rgbInf = cv2.cvtColor(np.asanyarray(infraredFrame), cv2.COLOR_GRAY2RGB)
    blank = np.zeros_like(rgbInf)
    blank[:,:,0].fill(0)
    blank[:,:,1].fill(255)
    blank[:,:,2].fill(255)
    coords = []
    if contours:
        #cv2.drawContours(blank, contours, -1, (255,255,0), 3)
        for cnt in contours:
            c = max(cnt, key=cv2.contourArea)
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            cv2.circle(blank, (int(x),int(y)), 2, (255,0,0), 1)
    [height,width] = np.array(depthFrame.shape)
    nx = np.linspace(0, width-1, width)
    ny = np.linspace(0, height-1, height)
    u, v = np.meshgrid(nx, ny)
    x = (u.flatten() - cameraIntrinsics['ppx'])/cameraIntrinsics['fx']
    y = (v.flatten() - cameraIntrinsics['ppy'])/cameraIntrinsics['fy']
    z = depthFrame.flatten() / 1000
    x = np.multiply(x,z)
    y = np.multiply(y,z)
    ##time2 = time.time()
    #x = x[np.nonzero(z)]
    #y = y[np.nonzero(z)]
    #z = z[np.nonzero(z)]

    rgbB = rgbInf[:,:,0].flatten().astype(int)
    rgbG = rgbInf[:,:,1].flatten().astype(int)
    rgbR = rgbInf[:,:,2].flatten().astype(int)
    rgbPC = rgbR<<16|rgbG<<8|rgbB
    rgbThres = 255<<16|0<<8|0
    ##time3=time.time()
    points = np.asanyarray([x,y,z])

    n = points.shape[1] 
    points_ = np.vstack((points, np.ones((1,n))))
    points_trans_ = np.matmul(poseMat, points_)
    points_transformed = np.true_divide(points_trans_[:3,:], points_trans_[[-1], :])
    ##time4=time.time()
    allPoints = np.asanyarray([points_transformed[0,:],points_transformed[1,:],points_transformed[2,:],rgbR/256, rgbG/256, rgbB/256]).T
    markerPoints = allPoints[allPoints[:,3]>0]
    return [allPoints, markerPoints]
    

In [63]:
filename = 'loco15walk1.pickle'
file = open(filename, 'rb')
file2 = open('loco15processed.pickle','wb')
#dataRead = []
#cloud = pcl.PointCloud_PointXYZRGBA()
#visual = pcl.pcl_visualization.CloudViewing()
#visual.setBackgroundColor (1.0, 0.5, 1.0)
#time.sleep(6)
#detector = cv2.SimpleBlobDetector_create()
#visual = pcl.pcl_visualization.CloudViewing()
i = 1
#interest = [93,97,100,105]
while 1:
    try:
        frame = pickle.load(file) #this loads each frame in one at a time, this is where we can process each frame with the transformation matrix
        #cloud = pcl.PointCloud_PointXYZRGBA()
        #if i in interest:
        pcd = PointCloud()
        mpcd = PointCloud()
        allPoints = np.empty((0,6))
        allmarkerPoints = np.empty((0,6))
        #for (serial, [poseMat, rmsdValue]) in self.devicesTransformation.items():
        for camera,deviceData in frame.items():
            depthFrame = deviceData['depth']
            infraredFrame = cv2.bitwise_or(deviceData['infrared'],deviceData['infrared'],bgFrames[camera])
            cameraIntrinsics = deviceData['intrinsics']
            poseMat = deviceData['poseMat']
            [points,markerPoints] = getDepthandMarkerPoints(infraredFrame,depthFrame,cameraIntrinsics,poseMat)
            ##markerPoint = depthFrametoPC(depthFilt, newImg, cameraIntrinsics, poseMat)
            #pclcolors = np.asanyarray(points[:,3]*256).astype('int')<<16|np.asanyarray(points[:,4]*256).astype('int')<<8|np.asanyarray(points[:,5]*256).astype('int')
            #rgbThres = 215<<16|215<<8|215
            #markerPoint = points[points[:,3]>0]
            allPoints = np.append(allPoints, points,axis=0)
            #allmarkerPoints = np.append(allmarkerPoints, markerPoints, axis=0)
        #cloud.from_array(allPoints.astype('float32'))
        pcd.points=Vector3dVector(allPoints[:,0:3])
        pcd.colors=Vector3dVector(allPoints[:,3:6])
        #mpcd.points=Vector3dVector(allmarkerPoints[:,0:3])
        #mpcd.colors=Vector3dVector(allmarkerPoints[:,3:6])
        #mpcl = open3d_to_pcl(mpcd)
        voxel_down_pcd = voxel_down_sample(pcd, voxel_size = 0.005)
        cl,ind = statistical_outlier_removal(voxel_down_pcd, nb_neighbors=40, std_ratio=2.0)
        pclPC = open3d_to_pcl(cl)
        
        for j in range(0,3):
            seg = pclPC.make_segmenter_normals(ksearch=50)
            seg.set_optimize_coefficients(True)
            seg.set_model_type(pcl.SACMODEL_NORMAL_PLANE)
            seg.set_method_type(pcl.SAC_RANSAC)
            seg.set_distance_threshold(0.005)
            seg.set_normal_distance_weight(0.003)
            seg.set_max_iterations(100)
            indices, coefficients = seg.segment()
            pclPC = pclPC.extract(indices, negative=True)
        #if indices != 0:
        
        #clFilt = pclPC.extract(indices, negative=True)
        finalPC = np.asarray(pclPC)
        pickle.dump(finalPC,file2)
        #mPoints = np.asarray(pclPC)
        #finalPCL = pcl.PointCloud_PointXYZRGBA()
        #finalPCL.from_array(np.append(finalPC,mPoints, axis=0).astype('float32'))
        #visual.ShowColorACloud(pclPC)
        print(i)
        i+=1
    except EOFError:
        break
file.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
def open3d_to_pcl(pointCloud):
    pclCloud = pcl.PointCloud_PointXYZRGBA()
    points = np.asarray(pointCloud.points)
    colors = np.asarray(pointCloud.colors)
    colors255 = np.asarray([[r*255,g*255,b*255] for r,g,b in colors])
    rgbPC = np.asarray([r.astype('int')<<16|g.astype('int')<<8|b.astype('int') for r,g,b in colors255]).reshape((-1, 1)) 
    pclFormat = np.hstack((points, rgbPC))
    rgbThres = 200<<16|200<<8|200
    pclFormat[np.where(pclFormat[:,3]>=rgbThres),:][0,:,:]
    pclCloud.from_array(pclFormat.astype('float32'))
    return pclCloud